<a href="https://colab.research.google.com/github/EssenceBL/MakingFriends/blob/main/Information_Retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Background**

One tells a tale to summer interns of the year to provide them with some relaxing moments.

--YK 2023, with assistance from Sage

**A Question on Information Retrieval**

One is given $n$ pieces of information, let's say $n$ numbers; and $2n$ distinguishable storage units, each of which can store a piece of information computed from the given $n$  numbers  . A robust scheme could tolerate damage in storing devices to certain extent. Could one define a storing scheme so that the origin $n$ pieces of information can be retrieved from any $n$ undamaged distinguishable storage units remain?

**Toy Cases**

The case when $n=1$ is trivial. We can just duplicate that number so that any one of the $2$ storage units contains the same origin information.

We need to think about the case when $n=2$. Let's begin with some numbers $a$ and $b$. The question didn't specify any number field; so it could be a Galois field, a prime field, the set of complex numbers, or maybe just the set of real numbers. Let's assume we are talking about the floating points for the time being for easy coding here.

To begin with, we are given $4$ distinguishable storage units $S_0,S_1,S_2,S_3$, each of which can store a number; let's say a floating point for the time being.



In [1]:
# @title Input 2 numbers
a = 11.0 # @param {type:"number"}
b = 15.0 # @param {type:"number"}
print(f"Given information a: {a}")
print(f"Given information b: {b}")

Given information a: 11.0
Given information b: 15.0


**Computation for Storage and Retrieval**

Now, we have a simple plan that might just work. Recall the graph of a non-vertical line can be given by its point-slope form $y=f(x)=a+bx$; and that this line is uniquely determined by any two points on it.

Then we can just assign to each storage $S_i=f(i)$ for $i=0,1,2,3$. For any two distinguishable storage units $S_i$ and $S_j$ where $i\neq j$, we can retrieve $a$ and $b$ by $b=\frac{S_i-S_j}{i-j}$ and $a=f(S_i)-ib$ accordingly.


In [2]:
import numpy as np
def compute(a,b,i):
  return a+b*i
supports = np.arange(4)
storage = np.vectorize(compute)(a, b, supports)

print(f"Example: we can compute under this storage scheme: {[[index, value] for index, value in enumerate(storage)]}")

Example: we can compute under this storage scheme: [[0, 11.0], [1, 26.0], [2, 41.0], [3, 56.0]]


Simulation on $2$ distinguishable storage units undamaged:

In [3]:
remains_i = np.random.choice(4, 2, replace=False)

#remains_i = np.array([1,3])

remains_e = storage[remains_i]

remains = np.concatenate((remains_i[:,None], remains_e[:,None]), axis=1)

print(remains)

[[ 1. 26.]
 [ 3. 56.]]


Simulation on original information retrieved:

In [4]:
def retrieve(remains):
  b = (remains[1][1]-remains[0][1])/(remains[1][0]-remains[0][0])
  a = remains[1][1] - remains[1][0]*b
  return a,b

print(f"Still, we can retrieve the original (a,b) here: {retrieve(remains)}")

Still, we can retrieve the original (a,b) here: (11.0, 15.0)


**The General Case**

In general, suppose we are given $n$ information  $a_0,a_1,...,a_{n-1}$ as follows:

In [5]:
import ast

info = "10,1,13,15,20,30" # @param {type: 'string'}
infoList = list(ast.literal_eval(str(info)))
n = len(infoList)
print(f"Example: given information: {infoList}")
print(f"Number of information n = {n}")

Example: given information: [10, 1, 13, 15, 20, 30]
Number of information n = 6


In [6]:
def computeS(info,i):
  infoList = list(ast.literal_eval(info))
  n = len(infoList)
  sum = 0
  for j in range(n):
    sum += infoList[j]*i**j
  return sum

Now, let's extend our storing scheme to a polynomial of degree $n-1$ with $n$ coefficients:
$$p(x)=\sum^{n-1}_{i=0} a_i x^i.$$
The storage scheme is just the assignment $S_i = p(i)$ for $i=0,1,...,2n-1$.

\[In the following, we print points of the graph of $[i,S_i]$ for $i=0,1,...,2n-1$. Please make sure previous cells are running properly before proceeding.\]

In [7]:
supportS = np.arange(2*n)
storageS = np.vectorize(computeS)(info, supportS)
print(f"Example: we compute under this storgae scheme as {[[index, value] for index, value in enumerate(storageS)]}")

Example: we compute under this storgae scheme as [[0, 10], [1, 89], [2, 1464], [3, 9445], [4, 37022], [5, 108465], [6, 262924], [7, 558029], [8, 1073490], [9, 1914697], [10, 3216320], [11, 5145909]]


Simulation on $n$ distinguishable storage units undamaged:

We print out the list of $[t_i, S_{t_i}]$ where  $t_i\neq t_j$ for $i\neq j$, $t_i\in \{0,1,...,2n-1\}$ and $i \in  \{0,1,...,n-1\}$.


In [8]:
remainS_i = np.random.choice(2*n, n, replace=False)
#remains_i = np.array([1,3,2,4,5,9])
remainS_e = storageS[remainS_i]
remainS = np.concatenate((remainS_i[:,None], remainS_e[:,None]), axis=1)
print(f"Suppose we have only half of the information remain: \n {remainS}.")

Suppose we have only half of the information remain: 
 [[      2    1464]
 [      0      10]
 [      7  558029]
 [      5  108465]
 [     10 3216320]
 [      6  262924]].


Now, given these $n$ undamaged points $S_{t_i}$ for $i = 0,1,...,n-1$, one can retrieve $p(x)$ by expanding the Lagrange polynomial; and accordingly, retrieve the original information by looking into its coefficients:


$$p(x) = \sum^{n-1}_{j=0} S_{t_j}\prod_{\substack{i\neq j\\ 0\leq i \leq n-1}} \frac{x-{t_i}}{ {t_j}  -{t_i}}.$$

In [9]:
from sympy import *
x = Symbol('x')
def retrieveS(remainS):
  n = len(remainS)
  sPs = [prod([(x - remainS[k][0])/(remainS[j][0]-remainS[k][0]) for k in range(n) if k != j]) for j in range(n)]
  P = sum([remainS[j][1]*sPs[j] for j in range(n)])
  return expand(P).as_coefficients_dict().values()
print(f"Still, one could retrieve the given infomation here: {str(retrieveS(remainS))}].")

Still, one could retrieve the given infomation here: dict_values([10, 1, 13, 15, 20, 30])].


**Remarks:**
1. In reality, Galois fields might be more commonly employed.
2. Further reading direction: you may search for error correction codes, such as Reed-Solomon codes.
3. The tale in this notebook assumes one know which storage units had been damaged and which ones remain well undamaged. If the damage positions are unknown and one wants to detect them as well, then one would need at least a bit more information. Like, given $2n+1$ storage units, one could make a scheme to restore from at most $n$ unknown storage units damaged. Indeed, we could again make use of a polynomial to make the storage scheme, and the recovery part would be like finding a best-fit solution of some overdetermined system in certain sense.